In [ ]:
# # Installations
# !pip install langchain-google-genai
# !pip install langchain
# !pip install -q -U google-generativeai
# !pip install chromadb
# !pip install PyPDF2

In [17]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import PyPDF2
import json
import os
import pandas as pd

In [18]:
credentials = open('/Users/bhavishyapandit/VSCProjects/security_for_llms/creds.json')
creds = json.load(credentials)

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file = '/Users/bhavishyapandit/VSCProjects/security_for_llms/data/auto-mpg.csv'
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3, google_api_key=creds['google'],
                            )

# Loading csv file
loader = CSVLoader(file_path=file, encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

# Splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
text_chunks = text_splitter.split_documents(data)
len(text_chunks)

398

In [20]:
from langchain_community.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=creds['google'])
vector_index = Chroma.from_documents(text_chunks, embeddings).as_retriever()

question = 'What is the acceleration of ford torino of model year 70'
docs = vector_index.get_relevant_documents(question)
print(docs[0].page_content)

mpg: 17.0
cylinders: 8
displacement: 302.0
horsepower: 140
weight: 3449
acceleration: 10.5
model year: 70
origin: 1
car name: ford torino
company name: ford


In [21]:
question = 'Which car has the highest displacement, tell me just the name.'
docs = vector_index.get_relevant_documents(question)
print(docs[0].page_content)

mpg: 12.0
cylinders: 8
displacement: 400.0
horsepower: 167
weight: 4906
acceleration: 12.5
model year: 73
origin: 1
car name: ford country
company name: ford


In [22]:
df = pd.read_csv(file)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,company name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino,ford


In [23]:
df.displacement.max()

455.0